# Processing of GLEIF data

In [ ]:
from collections import defaultdict
import csv

In [ ]:
def keep_first_two_columns(input_file, output_file):
    """Remove all columns of the input file except the first two columns and write the result to the output file"""
    # Open the input and output files
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        # Create CSV reader and writer
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # Read the rows of the input file and write only the first two columns to the output file
        for row in reader:
            writer.writerow(row[:2])

# # Remove all columns of the input file except the first two columns and write the result to the output file
# keep_first_two_columns('20230106-0800-gleif-goldencopy-lei2-golden-copy.csv', 'lei2-golden-copy-cleaned.csv')


In [ ]:
def keep_columns_one_three_five(input_file, output_file):
    """Remove all columns of the input file except the first, third and fifth columns and write the result to the output file"""
    # Open the input and output files
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        # Create CSV reader and writer
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # Read the rows of the input file and write only the first, third and fifth columns to the output file
        for row in reader:
            writer.writerow([row[0], row[2], row[4]])

# # Remove all columns of the input file except the first, third and fifth columns and write the result to the output file
# keep_columns_one_three_five('20230106-0800-gleif-goldencopy-rr-golden-copy.csv', 'rr-golden-copy-cleaned.csv')


In [ ]:
def merge_csvs(input1, input2, output):
    """Merge the two CSV files and write the output to a new CSV file"""
    # Open the input and output files
    with open(input1, 'r') as infile1, open(input2, 'r') as infile2, open(output, 'w', newline='') as outfile:
        # Create CSV readers and writer
        reader1 = csv.reader(infile1)
        reader2 = csv.reader(infile2)
        writer = csv.writer(outfile)

        # Create a mapping of company names to IDs from the first input file
        name_to_id = defaultdict(str)
        for row in reader1:
            name_to_id[row[0]] = row[1]

        # Seek back to the beginning of the first input file and reset the reader
        #infile1.seek(0)
        #reader1 = csv.reader(infile1)

        # Read the first row and modify the first two elements
        row = next(reader2)
        row = ['NameStart', 'NameEnd'] + row

        # Write the modified first row to the output file
        writer.writerow(row)

        # Read the rest of the rows of the second input file and look up the IDs in the mapping
        for row in reader2:
            writer.writerow([name_to_id[row[0]], name_to_id[row[1]]] + row)

# # Merge the two CSV files and write the output to a new CSV file
# merge_csvs('lei2-golden-copy-cleaned.csv', 'rr-golden-copy-cleaned.csv', 'rr-with-names.csv')



In [ ]:


def remove_duplicates(input_file, output_file):
    # Open the input file
    with open(input_file, 'r') as infile:
        # Create a CSV reader
        reader = csv.reader(infile)

        header = next(reader)

        # Read the rows of the input file and store them in a list
        reverse_reader = list(reader)

        # Reverse the order of the rows
        reverse_reader = reverse_reader[::-1]

        # Read the rows of the input file and store the unique values in a list
        rows = []
        seen = set()
        for row in reverse_reader:
            if row[0] not in seen:
                rows.append(row)
                seen.add(row[0])

        # Write the rows with the unique values to a new file
        with open(output_file, 'w', newline='') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(header)
            for row in rows:
                writer.writerow(row)

# # Remove duplicates from the input CSV file and write the result to a new CSV file
# remove_duplicates('rr-names-two-columns.csv', 'rr-clean.csv')


In [ ]:
def cleaning_pipeline(lei2_path, rr_path):
    """Performs the entire cleaning of the data from the raw source files to the final file that will interest us, rr-clean.csv"""
    keep_first_two_columns(lei2_path, 'lei2-golden-copy-cleaned.csv')
    keep_columns_one_three_five(rr_path, 'rr-golden-copy-cleaned.csv')
    merge_csvs('lei2-golden-copy-cleaned.csv', 'rr-golden-copy-cleaned.csv', 'rr-with-names.csv')
    keep_first_two_columns('rr-with-names.csv', 'rr-names-two-columns.csv')
    remove_duplicates('rr-names-two-columns.csv', 'rr-clean.csv')


Generation of the data that we will use for the rest of the project, cleaned and filtered

In [ ]:
cleaning_pipeline('20230106-0800-gleif-goldencopy-lei2-golden-copy.csv', '20230106-0800-gleif-goldencopy-rr-golden-copy.csv')

# Definition de la structure Trie et des differentes methodes associees

In [324]:
class Trie:
    def __init__(self):
        self.root = {}
        self.end_symbol = "*"

    def insert(self, company_name, parent_name):
        # Convert the company name to lowercase
        company_name = preprocess(company_name)
        # Start at the root of the trie
        current = self.root
        # Recalls our position in the company name
        pos = 0
        n = len(company_name)
        # Iterate over the letters in the company name
        for letter in company_name:
            # If the letter is not present in the current dictionary, add it with the given parent name
            if letter not in current:
                current[letter] = {"parent": parent_name, "distance_to_company" : n-pos-1, "closest_company" : company_name}
            # If the letter is present in the current dictionary and has a different parent name, set the parent name to None, and if it has the same parent name, update the distance to the smaller one
            elif "parent" in current[letter] and parent_name is not None:
                if current[letter]["parent"] != parent_name:
                    current[letter]["parent"] = None
                    current[letter]["distance_to_company"] = None
                    current[letter]["closest_company"] = None
                elif  n-pos-1 < current[letter]["distance_to_company"]:
                    current[letter]["distance_to_company"] = n-pos-1
                    current[letter]["closest_company"] = company_name
            # Move to the next letter
            pos += 1
            current = current[letter]
        # Add the end symbol with the company name and parent name
        current[self.end_symbol] = (company_name, parent_name)

    def search(self, company_name):
        # Convert the company name to lowercase
        company_name = preprocess(company_name)
        current = self.root
        for letter in company_name:
            if letter not in current:
                return None
            current = current[letter]
        if self.end_symbol in current:
            return current[self.end_symbol]
        return None

    def get_parent(self, path):
        # Start at the root of the trie
        current = self.root
        # Iterate over the letters in the path
        for letter in path:
            # If the letter is not present in the current dictionary, return None
            if letter not in current:
                return None
            # Move to the next letter
            current = current[letter]
        # If the parent field is present in the current node, return the parent company name
        if "parent" in current:
            return current["parent"]
        # If the parent field is not present, return None
        else:
            return None

    def get_distance(self, path):
        # Start at the root of the trie
        current = self.root
        # Iterate over the letters in the path
        for letter in path:
            # If the letter is not present in the current dictionary, return None
            if letter not in current:
                return None
            # Move to the next letter
            current = current[letter]
        # If the distance_to_company field is present in the current node, return the distance to the company that gave it its parent name
        if "distance_to_company" in current:
            return current["distance_to_company"]
        # If the distance_to_company field is not present, return None
        else:
            return None

    def get_closest_company(self, path):
        # Start at the root of the trie
        current = self.root
        # Iterate over the letters in the path
        for letter in path:
            # If the letter is not present in the current dictionary, return None
            if letter not in current:
                return None
            # Move to the next letter
            current = current[letter]
        # If the distance_to_company field is present in the current node, return the distance to the company that gave it its parent name
        if "closest_company" in current:
            return current["closest_company"]
        # If the distance_to_company field is not present, return None
        else:
            return None

    def find_nearest_with_parent(self, company_name):
        """Looks for the nearest node thas has a parent company.
        Returns a tuple composed of the path to the node, the parent name, the closest company name and the distane to the closest company"""
        # Convert the company name to lowercase
        company_name = preprocess(company_name)
        # Recursive function to search for closest leaf or node with parent company
        def nearest_aux(current, path, suffix_company_name, distance):
            # Initialize result with path and None for parent company
            # print(path, suffix_company_name, distance)
            res = path, None, None, distance
            # If suffix has more than one character, remove first and search child
            if len(suffix_company_name) > 0:
                # print('sfx', suffix_company_name)
                letter = suffix_company_name[0]
                new_suffix = suffix_company_name[1:]
                if letter in current:
                    res = nearest_aux(current[letter], path+letter, new_suffix, distance)
                    # If child does not have parent company, add to list of excluded children
                    # if res[1] is None:
                    #     to_exclude += [letter]
                    # else:
                    #     # print("res not None1", res)
                    #     return res
                    if res[1] is not None:
                        return res
            # If current is leaf with parent company, return company and parent
            elif self.end_symbol in current:
                res = path, current[self.end_symbol][1], current[self.end_symbol][0], distance
                if res[1] is not None:
                    # print("res not None2", res)
                    return res
            # If current is node with parent company, return path and parent
            elif "parent" in current and current['parent'] is not None:
                try:
                    distance += self.get_distance(path)
                except:
                    pass
                # print("res not None3", res)
                return path, current['parent'], current['closest_company'], distance
            # If nothing has been returned yet, it means that no parent company has been found, so we need to explore the other children
            for child in current:
                if  isinstance(child, str) and len(child) == 1:
                    # print(child)
                    new_path, parent, closest_company, new_distance = nearest_aux(current[child], path+child, '', distance + len(suffix_company_name)+ 1)
                    # print('child, parent', child, parent)
                    if parent is not None:
                        closest_company = self.get_closest_company(new_path)
                        # If self.get_distance(new_path) is None, we catch the exception
                        try:
                            res = new_path, parent, closest_company, new_distance + self.get_distance(new_path)
                        except:
                            res = new_path, parent, closest_company, new_distance
                        break
            # print("res not None4", res)
            return res
        return nearest_aux(self.root, '', company_name, 0)

    def update_with_children(self):
        """An update function that will update all of the tree nodes for the closest company among its descendants that has a parent company"""
        def dfs(current):
            # If the current node is a leaf with a parent company, update the parent, distance, and closest company fields
            if self.end_symbol in current and current[self.end_symbol][1] is not None:
                current["parent"] = current[self.end_symbol][1]
                current["distance_to_company"] = 0
                current["closest_company"] = current[self.end_symbol][0]
                return current["parent"], current["distance_to_company"], current["closest_company"]
            else:
                # Initialize variables for storing the closest parent company, distance to company, and closest company
                closest_parent, distance, closest_company = None, float("inf"), None
                # Iterate over the children of the current node
                for child in current:
                    if  isinstance(child, str) and len(child) == 1:
                        # print(child, current[child])
                        # Recursively search for the closest parent company, distance to company, and closest company in the child
                        child_parent, child_distance, child_company = dfs(current[child])
                        # If the child has a parent company and it is closer than the current closest parent company, update the closest parent company, distance to company, and closest company
                        if child_parent is not None and child_distance < distance:
                            closest_parent = child_parent
                            distance = child_distance
                            closest_company = child_company
                # If the closest parent company is not None, update the parent, distance, and closest company fields of the current node
                if closest_parent is not None:
                    current["parent"] = closest_parent
                    current["distance_to_company"] = distance + 1
                    current["closest_company"] = closest_company
                    return current["parent"], current["distance_to_company"], current["closest_company"]
            return None, float("inf"), None
        dfs(self.root)

    def update_with_nearest(self):
        """An update function that will update all of the tree nodes for the closest company by looking at nearest neighbors (not limited to descendants)"""

        # Define a helper function to perform the DFS
        def dfs(current, path):
            # If the current node is a leaf, find the nearest company with a parent and update the fields
            if self.end_symbol in current:
                nearest_path, parent, closest_company, distance = self.find_nearest_with_parent(path)
                current["parent"] = parent
                current["distance_to_company"] = distance
                current["closest_company"] = closest_company
                return current["parent"], current["distance_to_company"], current["closest_company"]

            # If the current node is not a leaf, update the fields for each child node
            else:
                # Initialize variables for storing the closest parent company, distance to company, and closest company
                closest_parent, distance, closest_company = None, float("inf"), None

                for child in current:
                    if  isinstance(child, str) and len(child) == 1:
                        child_parent, child_distance, child_company = dfs(current[child], path + child)

                        if child_parent is not None and child_distance < distance:
                            closest_parent = child_parent
                            distance = child_distance
                            closest_company = child_company

                # If the closest parent company is not None, update the parent, distance, and closest company fields of the current node
                if closest_parent is not None:
                    current["parent"] = closest_parent
                    current["distance_to_company"] = distance + 1
                    current["closest_company"] = closest_company
                    return current["parent"], current["distance_to_company"], current["closest_company"]
            return None, float("inf"), None

        # Start the DFS at the root of the trie
        dfs(self.root, '')

    def update(self):
        """A complete update of the trie, beginning by a complete update looking at descendants exclusively, to then update taking into account other neighbors"""
        # We first update every node of the trie to get the parent company of its closest descendant that has a parent company
        _ = self.update_with_children()
        # We then broaden our scope to allow for a node to have for parent company the parent company of its closest neighbor that has a parent company, even though it is not of its descendants. By doing so in this order, we make sure that the order of insertion does not matter in the final result
        _ = self.update_with_nearest()
        return self

    def insert_dataframe(self, df, name_column, parent_column):
        # Iterate over the rows of the dataframe
        for _, row in df.iterrows():
            # Extract company name and parent company from row
            company_name = row[name_column]
            parent_name = row[parent_column]
            # Insert company into trie
            try:
                self.insert(company_name, parent_name)
            except:
                pass
                # print(company_name, parent_name)
        self.update()


def trie_to_dataframe(trie):
    data = []

    def dfs(node):
        if trie.end_symbol in node:
            data.append((node[trie.end_symbol][0], node[trie.end_symbol][1]))
        for child in node:
            if  isinstance(child, str) and len(child) == 1:
                dfs(node[child])

    dfs(trie.root)
    df = pd.DataFrame(data, columns=["Company", "Parent"])
    return df


# Test des differentes fonctionnalites de la structure Trie

In [380]:
trie = Trie()
trie.insert("Apple Retail", "Apple")
trie.insert("Linkedin", "Microsoft")

node = trie.root["a"]
print(node["parent"])

node = trie.root["l"]
print(node["parent"])


Apple
Microsoft


We look at the structure of the trie with the different fields at every node : parent, closest_company, distance_to_company

In [381]:
trie.root

{'a': {'parent': 'Apple',
  'distance_to_company': 13,
  'closest_company': 'apple payments',
  'p': {'parent': 'Apple',
   'distance_to_company': 12,
   'closest_company': 'apple payments',
   'p': {'parent': 'Apple',
    'distance_to_company': 11,
    'closest_company': 'apple payments',
    'l': {'parent': 'Apple',
     'distance_to_company': 10,
     'closest_company': 'apple payments',
     'e': {'parent': 'Apple',
      'distance_to_company': 9,
      'closest_company': 'apple payments',
      ' ': {'parent': 'Apple',
       'distance_to_company': 8,
       'closest_company': 'apple payments',
       'p': {'parent': 'Apple',
        'distance_to_company': 7,
        'closest_company': 'apple payments',
        'a': {'parent': 'Apple',
         'distance_to_company': 6,
         'closest_company': 'apple payments',
         'y': {'parent': 'Apple',
          'distance_to_company': 5,
          'closest_company': 'apple payments',
          'm': {'parent': 'Apple',
           'dist

In [382]:
trie.insert("Apple Payments", None)
trie.update()
result = trie.find_nearest_with_parent("Apple Payments")
print(result)

('apple payment', 'Apple', 'apple payments', 15)


In [385]:
trie.insert("Apple Juice", "Innocent Inc")
result = trie.find_nearest_with_parent("Apple Payments")
print(result)

('apple payment', 'Apple', 'apple payments', 15)


In [ ]:
# Create a Trie object
trie = Trie()

# Insert some elements into the trie
trie.insert("Apple", "Fruit company")
trie.insert("Applebee's", "Dining company")
trie.insert("Microsoft", "Technology company")
trie.insert("Microchip", "Chip company")
trie.insert("Micro", None)

# Search for closest leaf or node with parent company
result = trie.search("Micro")
print(result)

# Search for closest leaf or node with parent company
result = trie.find_nearest_with_parent("Micro")
print(result)

In [ ]:
trie.root

In [388]:
# Create a Trie object
trie = Trie()

# Create a dataframe with two columns: "Company" and "Parent"
data = {
    "Company": ["Apple India", "Apple Juice", "Microsoft", "Microchip", "Micro"],
    "Parent": ["Apple", "Innocent", "Microsoft", "ST Microlectronics", None]
}
df = pd.DataFrame(data)

# Insert rows of dataframe into trie
trie.insert_dataframe(df, 'Company', 'Parent')

trie.root

{'a': {'parent': 'Apple',
  'distance_to_company': 10,
  'closest_company': 'apple india',
  'p': {'parent': 'Apple',
   'distance_to_company': 9,
   'closest_company': 'apple india',
   'p': {'parent': 'Apple',
    'distance_to_company': 8,
    'closest_company': 'apple india',
    'l': {'parent': 'Apple',
     'distance_to_company': 7,
     'closest_company': 'apple india',
     'e': {'parent': 'Apple',
      'distance_to_company': 6,
      'closest_company': 'apple india',
      ' ': {'parent': 'Apple',
       'distance_to_company': 5,
       'closest_company': 'apple india',
       'i': {'parent': 'Apple',
        'distance_to_company': 4,
        'closest_company': 'apple india',
        'n': {'parent': 'Apple',
         'distance_to_company': 3,
         'closest_company': 'apple india',
         'd': {'parent': 'Apple',
          'distance_to_company': 2,
          'closest_company': 'apple india',
          'i': {'parent': 'Apple',
           'distance_to_company': 1,
         

In [389]:
df = trie_to_dataframe(trie)
print(df)

       Company              Parent
0  apple india               Apple
1  apple juice            Innocent
2        micro                None
3    microsoft           Microsoft
4    microchip  ST Microlectronics


# Travail avec `mapping df` et structure Trie

In [392]:
import pandas as pd
from collections import defaultdict

df = pd.read_csv('Case_study_names_mapping.csv')
mapping_df = pd.read_csv('rr-clean.csv')

mapping_df

,NameStart,NameEnd
0,GE Holdings Luxembourg & Co S.à.r.l.,GENERAL ELECTRIC COMPANY
1,INSIGHT INVESTMENT INTERNATIONAL LIMITED,THE BANK OF NEW YORK MELLON CORPORATION
2,Managed Fund / Tiedemann Arbitrage Enhanced Fu...,SG KLEINWORT HAMBROS CORPORATE SERVICES (CI) L...
3,CHEYNE SELECT UCITS FUND PUBLIC LIMITED COMPANY,CHEYNE CAPITAL SMC LIMITED
4,INGREDION UK LIMITED,INGREDION INCORPORATED
...,...,...
209788,III SELECT CREDIT HUB FUND LTD.,III Capital Management
209789,GESTION PLACEMENTS DESJARDINS INC.,FÉDÉRATION DES CAISSES DESJARDINS DU QUÉBEC
209790,GE Aviation Czech s.r.o.,GENERAL ELECTRIC COMPANY
209791,Oakmark International Fund,HARRIS ASSOCIATES INVESTMENT TRUST


In [391]:
mapping_trie = Trie()
mapping_trie.insert_dataframe(mapping_df, 'NameStart', 'NameEnd')
# mapping_trie

In [393]:
result = mapping_trie.search('abaques')
print(result)

('abaques', 'THE HOWARD GAMBSON COMPANY')


In [399]:
def add_closest_match(trie, df):
    """Updates a dataframe with the information contained in the trie, looking for the closest match in the trie for each company in the dataframe"""
    # Initialize new columns in the dataframe
    df['Closest node'] = None
    df['Closest company'] = None
    df['Distance to closest'] = None
    df['Closest parent'] = None

    for index, row in df.iterrows():
        company_name = row['Raw name']
        company_name = preprocess(company_name)
        # Find the closest match in the trie
        path, parent, closest_company, distance = trie.find_nearest_with_parent(company_name)
        # Update the row with the closest matches and distance
        df.at[index, 'Closest node'] = path
        df.at[index, 'Closest parent'] = parent
        df.at[index, 'Distance to closest'] = distance
        df.at[index, 'Closest company'] = closest_company

    return df

df = pd.read_csv('Case_study_names_mapping.csv')

add_closest_match(mapping_trie, df)

,Raw name,Mapped name,Closest node,Closest company,Distance to closest,Closest parent
0,"""ACCESOS NORMALIZADOS, SL""",NaN,accesor,accesor aplications and services,88,JCM TECHNOLOGIES SA
1,"""ALTAIX ELECTRONICA , S.A.L.""",NaN,altair eu,altair european opportunities fi,136,"ALTAIR FINANCE ASSET MANAGEMENT SGIIC, SA"
2,"""ANTALA LOCKS & ACCESORIS, SL""",NaN,antalia,antalia,19,JAD
3,"""ANTERAL, SL""",NaN,antero,antero peak master fund lp,43,ARTISAN PARTNERS LIMITED PARTNERSHIP
4,"""ARQUIMEA INGENIERIA , S.L.""",NaN,arquimea ingenieria,arquimea ingenieria,0,ARQUIMEA ENTERPRISES SL
...,...,...,...,...,...,...
4890,TFN PROPROTÃÂ,NaN,tfs,None,14,VIEL ET COMPAGNIE-FINANCE
4891,UK AND IRELAND SAP USER GROUP LTD,NaN,uk as,uk assistance,42,DIRECT LINE INSURANCE GROUP PLC
4892,VSW VERBAND FUER SICHERHEIT IN DER WIRTSCHAFT,NaN,vsws,vswsl masterfonds,64,UI BVK Kapitalverwaltungsgesellschaft mbH
4893,WILLI EBERLEIN GMBH,NaN,william,None,11,SOCIETE GENERALE


## Filtering common words

In [395]:
import pandas as pd
import re
from collections import Counter

def get_frequent_words(df, column, num_words=10, to_remove=[]):
    # Convert the column values to lowercase and split them into a list of words
    words = df[column].str.lower().str.split()
    # Flatten the list of words into a single list
    all_words = [word for sublist in words for word in sublist]
    # Remove from the count the words in to_remove and single characters to let the user focus on frequent unexpected words
    filtered_words = [word for word in all_words if (word not in to_remove and len(word) > 1)]
    # Count the frequency of each word
    word_counts = Counter(filtered_words)
    # Get the num_words most common words
    most_common = word_counts.most_common(num_words)
    # Return a list of the most common words
    return [word for word, count in most_common], word_counts

# frequent_words, word_counts = get_frequent_words(df, "Raw name")
# print(frequent_words)

Below, we are choosing the strings not to take into account in our comparison, among the most frequent strings that have not yet been identified as irrelevant

In [277]:
import ipywidgets as widgets
from IPython.display import display
import csv

def get_words_to_filter(primary_file, secondary_file, n = 20):
    try:
        with open(primary_file, 'r') as f:
            # If there is a predefined list of words not to take into account in our name comparisons, import it into the to_remove list
            reader = csv.reader(f)
            primary_list = list(reader)[0]
    except (FileNotFoundError, IndexError):
        # If the file does not exist or is empty, create an empty primary_list
        primary_list = []

    try:
        with open(secondary_file, 'r') as f:
            # If there is a predefined list of words not to take into account in our name comparisons, import it into the to_remove list
            reader = csv.reader(f)
            secondary_list = list(reader)[0]
    except (FileNotFoundError, IndexError):
        # If the file does not exist or is empty, create an empty secondary_list
        secondary_list = []

    frequent_words, _ = get_frequent_words(df, "Raw name", n, primary_list + secondary_list)

    return frequent_words, primary_list, secondary_list

def save_words_to_filter(filename, strings_list):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(strings_list)

def reset_word_files(file1, file2):
    with open(file1, 'w') as f:
        pass
    with open(file2, 'w') as f:
        pass


### First pass

In this pass, we prepare the filtering of words such as suffixes of organizations, country names and conjunctions of coordination to make a first filtering.
These words are similar in that they hold no discriminative power whatsoever (or at least it is extremely unlikely they hold power to help us group organizations together)

In [262]:
frequent_words, to_remove, _ = get_words_to_filter('irrelevant_words.csv', 'indiscriminative_words.csv', n=100)

menu = widgets.SelectMultiple(
    options=frequent_words,
    value=[],
    #rows=10,
    description='Parasites',
    disabled=False
)

display(menu)


SelectMultiple(description='Parasites', options=('altran', 'atos', 'aim', 'alten', 'ace', 'ass', 'august', 'co…

In [ ]:
# menu.value

In [263]:
to_remove += list(menu.value)

save_words_to_filter("irrelevant_words.csv", to_remove)

print(to_remove)


['&', 'ltd', 'de', 'gmbh', 'association', 'co', 'kg', 'inc', 'sl', '-', 'sa', 'sas', 'limited', 'und', 'a', 'llc', 'sarl', 'ag', 'fuer', 'et', 'la', 'of', 'inc.', 'ab', 'company', 'co.', 'asoc', 'as', 'arbeitsgemeinschaft', 'bv', 's', 's.l.', 'l', 's.a.', 'slu', 'f', 'e', 'llp', 'corp', 'gbr', 'v', '+', 'cokg', 'ltd.', 'd', 'pty', 'srl', 'al', 'z', 'm', 'r', 'b.v.', 'ac', '(uk)']


### Second pass

In this pass, we prepare the filtering words associated for instance to the type of business ('technologies', 'autohaus'...) , as having only such words in common cannot justify pairing two organizations. The user is free to remove any word that fit in this category (not able to discriminate alone on the proximity of two organizations but still holding meaning).

In [264]:
frequent_words, to_remove_indiscriminative, _ = get_words_to_filter('indiscriminative_words.csv', 'irrelevant_words.csv', n=100)


menu = widgets.SelectMultiple(
    options=frequent_words,
    value=[],
    #rows=10,
    description='Parasites',
    disabled=False
)

display(menu)

SelectMultiple(description='Parasites', options=('altran', 'atos', 'aim', 'alten', 'ace', 'ass', 'august', 'co…

In [396]:
# menu.value

In [265]:
to_remove_indiscriminative += list(menu.value)
save_words_to_filter("indiscriminative_words.csv", to_remove_indiscriminative)
print(to_remove_indiscriminative)

['autohaus', 'services', 'group', 'france', 'technologies', 'systems', 'consulting', 'solutions', 'advanced', 'asociacion', 'engineering', 'akademie', 'des', 'service', 'hotel', 'ev', 'akka', 'uk', 'international', 'american', 'andreas', 'atlas', 'technology', 'agence', 'management', 'y', 'and', 'industrie', 'software', 'atelier', 'europe', 'auto', 'deutschland', 'alfred', 'atlantic', 'ouest', 'ateliers', 'der', 'industrial', 'business', 'design', 'apleona', 'espanola', 'fã\x9cr', 'germany', 'du', 'alliance', 'formation', 'atlantique', 'servicios', 'abc', 'albert', 'all', 'private', 'media', 'center', 'spain', 'events', 'active', 'gesellschaft', 'en', 'arbeit', 'am', 'arnold', 'art', 'corporation', 'automation', 'sud', 'adolf', 'ambassade', 'products', 'industries', 'integration', 'control', 'francaise', 'applied', 'power', 'technical', 'partners', 'security', 'conseil', 'training', 'academy', 'information', 'systemes', 'groupe', 'it', 'audio', 'express', 'informatique', 'investigacion

## Utilities

In [398]:
import string

def preprocess(s, to_remove = to_remove):
    if not isinstance(s, str):
         try:
             s = str(s)
         except:
             return ''
    # Convert to lowercase
    s = s.lower()

    # Remove all special characters except for &, but preserve spaces
    for c in string.punctuation:
        if c != '&':
            s = s.replace(c, '')

    # Split into a list of words
    words = s.split()

    # Remove words in the to_remove list
    words = [word for word in words if word not in to_remove]

    # Join the words back into a single string
    s = ' '.join(words)

    return s


preprocess('sjf,d sdkF&sd fdf')

'sjfd sdkf&sd fdf'

In [371]:
import numpy as np

def longest_common_subsequence(s1, s2):
    """Dynamic Programming implementation of the longest common subsequence problem"""
    # Create a table to store results of sub-problems
    m, n = len(s1), len(s2)
    X = [[0] * (n + 1) for _ in range(m + 1)]

    # Iterate through s1 and s2, filling in the table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                X[i][j] = X[i - 1][j - 1] + 1
            else:
                X[i][j] = max(X[i - 1][j], X[i][j - 1])

    # Follow the path traced by the table to recover the LCS itself
    i, j, index = m, n, X[-1][-1]
    lcs = [""] * index

    while index > 0:
        if s1[i - 1] == s2[j - 1]:
            lcs[index - 1] = s1[i - 1]
            i -= 1
            j -= 1
            index -= 1
        elif X[i - 1][j] > X[i][j - 1]:
            i -= 1
        else:
            j -= 1

    return "".join(lcs)


def longest_common_substring_of_list(strings_list):

    # Determine size of the array
    n = len(strings_list)

    # Take first word from array
    # as reference
    s = strings_list[0]
    l = len(s)

    res = ""

    for i in range(l):
        for j in range(i + 1, l + 1):

            # generating all possible substrings of our reference string arr[0] i.e s
            stem = s[i:j]
            #k = 1
            for k in range(1, n):

                # Check if the generated stem is common to all words
                if stem not in strings_list[k]:
                    break

                # If current substring is present in all strings and its length is greater than current result
                if (k + 1 == n and len(res) < len(stem)):
                    #print('in', stem)
                    res = stem
    return res

## Cleaning dataframe and attributing a score

In [408]:
def closeness_score(row, h1, h2, h3, clustering=False):
    # Preprocess the raw name and the closest node, company, and parent fields
    raw_name, closest_node, closest_company, closest_parent = preprocess(row["Raw name"]), preprocess(row["Closest node"]), preprocess(row["Closest company"]), preprocess(row["Closest parent"])
    names = [raw_name, closest_node, closest_company, closest_parent]
    if clustering:
        clustering_parent = preprocess(row["LCS clustering parent"])
    n = len(raw_name)

    # Split the different fields fields into sets of words
    raw_name_words = set(raw_name.split())
    words = []
    for name in names:
        words += [set(name.split()) if name is not None else set()]
    if clustering:
        clustering_parent_words = set(clustering_parent.split()) if clustering_parent is not None else set()

    # Find the common words between all sets
    common_words = list(set.intersection(*words))

    # Find the longest common subsequence between the raw name and each of the closest node, company, and parent fields
    lcs_list = []
    for name in names:
        try:
            lcs_list += [longest_common_subsequence(raw_name, name).strip() if name is not None else '']
        except AttributeError:
            print(name)

    llcs_length = max([len(lcs) for lcs in lcs_list])

    # Normalize the longest common subsequence length by dividing it by the length of the the raw name
    normalized_llcs_length = llcs_length/n if n!=0 else 0

    # Calculate the difference between the length of the longest common subsequence and the length of the longest common word
    difference = llcs_length - max([len(word) for word in common_words]) if common_words else 0

    # Calculate the score as a function of the length of the longest common subsequence, the normalized longest common subsequence length, and the number of common words
    score = h1*np.log(1 + llcs_length) + h2*normalized_llcs_length + h3*len(common_words)

    # The parent we currently consider is the gleif extracted one. If the cluster inferred one proves to be better,
    # this will be updated
    what_parent = 'gleif'

    # In the case we are inspectig data after clustering, we do the same as earlier but with the LCS clustering parent
    if clustering:
        common_words_clustering = list(words[0] & clustering_parent_words)
        lcs_clustering = longest_common_subsequence(raw_name, clustering_parent).strip() if name is not None else ''
        llcs_length_clustering = len(lcs_clustering)
        normalized_llcs_length_clustering = llcs_length_clustering/n if n!=0 else 0
        difference_clustering = llcs_length - max([len(word) for word in common_words_clustering]) if common_words_clustering else 0
        score_clustering = h1*np.log(1 + llcs_length_clustering) + h2*normalized_llcs_length_clustering + h3*len(common_words_clustering)
        if score_clustering > score:
            score = score_clustering
            llcs_length = llcs_length_clustering
            normalized_llcs_length = normalized_llcs_length
            difference = difference_clustering
            common_words = common_words_clustering
            # row["Closest parent"] = np.nan
            what_parent = 'cluster'
        # else:
        #     # row["LCS clustering parent"] = np.nan
        #     what_parent = 'gleif'

    # Return a dictionary with the calculated score, the length of the longest common subsequence, the normalized longest common subsequence length, the difference between the length of the longest common subsequence and the length of the longest common word, and the list of common words
    return {'score' : score,'llcs_length' : llcs_length, 'normalized_llcs_length' : normalized_llcs_length, 'difference' : difference, 'common_words' : common_words, 'what_parent' : what_parent}


def clean_row(row, to_remove, to_remove_indiscriminative, h1, h2, h3, difference_threshold = 2, clustering=False):
    # Calculate scores for the row based on common words and longest common subsequences
    scores = closeness_score(row, h1, h2, h3, clustering)
    common_words = scores['common_words']

    # If there are no common words between Raw name and any of the other fields, or if the only common word is a single character or a word in the to_remove list, set the fields to NaN
    # if not common_words or (len(common_words)==1 and (len(common_words[0]) == 1 or common_words[0] in to_remove)):
    #     # print('not common_words', scores)
    #     if not clustering:
    #
    #         row["Closest company"], row["Distance to closest"], row["Closest parent"] = np.nan, np.nan, np.nan
    #     if clustering:
    #         row['LCS clustering parent'] = np.nan
    # else:

    # If there are no common words between Raw name and any of the other fields, or if the only common word is a single character
    # or a word in the to_remove list, we do not change the "What parent" field. Else (the case that follows), we change the
    # 'What parent' field to scores['what_parent']
    if common_words and not (len(common_words)==1 and (len(common_words[0]) == 1 or common_words[0] in to_remove)):
        # If there are common words that are also in the to_remove_indiscriminative list, check the difference between the length of the longest common subsequence and the length of the longest common word
        if any(word in to_remove_indiscriminative for word in common_words):
            # print('indiscriminative', scores)
            # If the difference is less than the threshold, set the fields to NaN. This is because the common word cannot discriminate by itself so having the common words very close to the lcs suggests that we don't have much more information to decide the companies are really linked. Another approach would have been to include a penalty in the score.
            if scores['difference'] > difference_threshold:
                row['What parent'] = scores['what_parent']
            #
            # if scores['difference'] < difference_threshold:
            #     row["Closest company"], row["Distance to closest"], row["Closest parent"] = np.nan, np.nan, np.nan
            #     if clustering:
            #         row['LCS clustering parent'] = np.nan
        else:
            # print('ok', scores)
            # Otherwise, set the Confidence score for the row
            # row['Confidence score'] = scores['score']
            row['What parent'] = scores['what_parent']
    return row

In [401]:
h1, h2, h3 = 1, 5, 2 # For the time being these are arbitrary values but these parameters can be learned later, with a machine learning algorithm for instance or an optimization algorithm
df_clean = df.copy()
df_clean['Confidence score'] = None
df_clean['What parent'] = None
df_clean = df_clean.apply(clean_row, axis=1, args=(to_remove, to_remove_indiscriminative, h1, h2, h3))
df_clean

,Raw name,Mapped name,Closest node,Closest company,Distance to closest,Closest parent,Confidence score,What parent
0,"""ACCESOS NORMALIZADOS, SL""",NaN,accesor,accesor aplications and services,88,JCM TECHNOLOGIES SA,None,None
1,"""ALTAIX ELECTRONICA , S.A.L.""",NaN,altair eu,altair european opportunities fi,136,"ALTAIR FINANCE ASSET MANAGEMENT SGIIC, SA",None,None
2,"""ANTALA LOCKS & ACCESORIS, SL""",NaN,antalia,antalia,19,JAD,None,None
3,"""ANTERAL, SL""",NaN,antero,antero peak master fund lp,43,ARTISAN PARTNERS LIMITED PARTNERSHIP,None,None
4,"""ARQUIMEA INGENIERIA , S.L.""",NaN,arquimea ingenieria,arquimea ingenieria,0,ARQUIMEA ENTERPRISES SL,None,gleif
...,...,...,...,...,...,...,...,...
4890,TFN PROPROTÃÂ,NaN,tfs,None,14,VIEL ET COMPAGNIE-FINANCE,None,None
4891,UK AND IRELAND SAP USER GROUP LTD,NaN,uk as,uk assistance,42,DIRECT LINE INSURANCE GROUP PLC,None,None
4892,VSW VERBAND FUER SICHERHEIT IN DER WIRTSCHAFT,NaN,vsws,vswsl masterfonds,64,UI BVK Kapitalverwaltungsgesellschaft mbH,None,None
4893,WILLI EBERLEIN GMBH,NaN,william,None,11,SOCIETE GENERALE,None,None


In [295]:
df

,Raw name,Mapped name,Closest node,Closest company,Distance to closest,Closest parent
0,"""ACCESOS NORMALIZADOS, SL""",NaN,accesor,accesor aplications and services sa,96,JCM TECHNOLOGIES SA
1,"""ALTAIX ELECTRONICA , S.A.L.""",NaN,altair eu,altair european opportunities fi,133,"ALTAIR FINANCE ASSET MANAGEMENT SGIIC, SA"
2,"""ANTALA LOCKS & ACCESORIS, SL""",NaN,antalia,antalia,17,JAD
3,"""ANTERAL, SL""",NaN,antero,antero peak master fund lp,43,ARTISAN PARTNERS LIMITED PARTNERSHIP
4,"""ARQUIMEA INGENIERIA , S.L.""",NaN,arquimea,arquimea group sa,27,ARQUIMEA ENTERPRISES SL
...,...,...,...,...,...,...
4890,TFN PROPROTÃÂ,NaN,tfs,tfs sa,17,VIEL ET COMPAGNIE-FINANCE
4891,UK AND IRELAND SAP USER GROUP LTD,NaN,ukad,ukad,22,ERAMET
4892,VSW VERBAND FUER SICHERHEIT IN DER WIRTSCHAFT,NaN,vsws,vswslmasterfonds,57,UI BVK Kapitalverwaltungsgesellschaft mbH
4893,WILLI EBERLEIN GMBH,NaN,william,None,10,SOCIETE GENERALE


# Traitement des organisations isolees
Pour lesquelles on n'a pas pu obtenir d'information dans la base de donnees

In [ ]:
mapping_trie.insert_dataframe(df_clean, 'Raw name', 'Closest parent')

In [402]:
def add_first_word_column(df):
    """Adds a column 'First word to the dataframe and fills it with the first word of the company name"""
    # Create an empty list to store the first words
    first_words = []

    # Iterate over the 'Raw name' column
    for name in df['Raw name']:
        # Split the name into words
        words = name.split()
        # Get the first word
        first_word = words[0] if len(words) > 0 else ''
        # Add the first word to the list
        first_words.append(first_word)

    # Add the 'First word' column to the dataframe
    df['First word'] = first_words

    return df

In [403]:
from sklearn.cluster import AffinityPropagation
from Levenshtein import distance

def cluster_update(df):
    """"This function updates a dataframe by performing affinity propagation clustering on the 'Raw name' column,
    grouping the words by cluster, and finding the longest common substring for each cluster
    to add to the 'LCS clustering parent' column."""

    def group_by_cluster(chunk, clusters):
        # Map each word to its cluster label
        cluster_map = {word: cluster for word, cluster in zip(chunk, clusters)}

        # Group the words by cluster label
        cluster_groups = {}
        for word, cluster in cluster_map.items():
            if cluster not in cluster_groups:
                cluster_groups[cluster] = []
            cluster_groups[cluster].append(word)

        return cluster_groups

    df['LCS clustering parent'] = None

    # Group the dataframe by 'Common word'
    grouped_df = df.groupby('First word')

    # Iterate over the groups and extract the 'Raw name' column
    chunks = []
    # results = []
    for common_word, group in grouped_df:
        chunk = group['Raw name'].tolist()
        chunks.append(chunk)

        # Compute the distance matrix
        n = len(chunk)
        distance_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                distance_matrix[i, j] = distance(chunk[i], chunk[j])

        # Perform affinity propagation clustering
        af = AffinityPropagation(damping=0.5).fit(distance_matrix)
        clusters = af.labels_

        cluster_groups = group_by_cluster(chunk, clusters)

        clustering_parent_map = {}

        for cluster, words in cluster_groups.items():
            lcs_cluster = longest_common_substring_of_list(words)
            for word in words:
                clustering_parent_map[word] = lcs_cluster

        group['LCS clustering parent'] = group['Raw name'].apply(lambda x : clustering_parent_map[x])

        # Update the original dataframe with the modified values in the group
        df.update(group)

    return df

In [404]:
df_cluster = add_first_word_column(df_clean)
df_cluster = cluster_update(df_cluster)

df_cluster

/Users/hish/miniconda3/envs/d3s/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:144: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
/Users/hish/miniconda3/envs/d3s/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:144: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
/Users/hish/miniconda3/envs/d3s/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:144: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
/Users/hish/miniconda3/envs/d3s/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:144: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
/Users/hish/miniconda3/envs/d3s/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:144: UserW

,Raw name,Mapped name,Closest node,Closest company,Distance to closest,Closest parent,Confidence score,What parent,First word,LCS clustering parent
0,"""ACCESOS NORMALIZADOS, SL""",NaN,accesor,accesor aplications and services,88.0,JCM TECHNOLOGIES SA,None,None,"""ACCESOS",
1,"""ALTAIX ELECTRONICA , S.A.L.""",NaN,altair eu,altair european opportunities fi,136.0,"ALTAIR FINANCE ASSET MANAGEMENT SGIIC, SA",None,None,"""ALTAIX",
2,"""ANTALA LOCKS & ACCESORIS, SL""",NaN,antalia,antalia,19.0,JAD,None,None,"""ANTALA",
3,"""ANTERAL, SL""",NaN,antero,antero peak master fund lp,43.0,ARTISAN PARTNERS LIMITED PARTNERSHIP,None,None,"""ANTERAL,",
4,"""ARQUIMEA INGENIERIA , S.L.""",NaN,arquimea ingenieria,arquimea ingenieria,0.0,ARQUIMEA ENTERPRISES SL,None,gleif,"""ARQUIMEA",
...,...,...,...,...,...,...,...,...,...,...
4890,TFN PROPROTÃÂ,NaN,tfs,None,14.0,VIEL ET COMPAGNIE-FINANCE,None,None,TFN,TFN
4891,UK AND IRELAND SAP USER GROUP LTD,NaN,uk as,uk assistance,42.0,DIRECT LINE INSURANCE GROUP PLC,None,None,UK,
4892,VSW VERBAND FUER SICHERHEIT IN DER WIRTSCHAFT,NaN,vsws,vswsl masterfonds,64.0,UI BVK Kapitalverwaltungsgesellschaft mbH,None,None,VSW,
4893,WILLI EBERLEIN GMBH,NaN,william,None,11.0,SOCIETE GENERALE,None,None,WILLI,


In [409]:
h1, h2, h3 = 1, 5, 2 # For the time being these are arbitrary values but these parameters can be learned later, with a machine learning algorithm for instance or an optimization algorithm
df_cluster_clean = df_cluster.copy()
df_cluster_clean = df_cluster_clean.apply(clean_row, axis=1, args=(to_remove, to_remove_indiscriminative, h1, h2, h3, 2, True))
df_cluster_clean

,Raw name,Mapped name,Closest node,Closest company,Distance to closest,Closest parent,Confidence score,What parent,First word,LCS clustering parent
0,"""ACCESOS NORMALIZADOS, SL""",NaN,accesor,accesor aplications and services,88.0,JCM TECHNOLOGIES SA,None,None,"""ACCESOS",
1,"""ALTAIX ELECTRONICA , S.A.L.""",NaN,altair eu,altair european opportunities fi,136.0,"ALTAIR FINANCE ASSET MANAGEMENT SGIIC, SA",None,None,"""ALTAIX",
2,"""ANTALA LOCKS & ACCESORIS, SL""",NaN,antalia,antalia,19.0,JAD,None,None,"""ANTALA",
3,"""ANTERAL, SL""",NaN,antero,antero peak master fund lp,43.0,ARTISAN PARTNERS LIMITED PARTNERSHIP,None,None,"""ANTERAL,",
4,"""ARQUIMEA INGENIERIA , S.L.""",NaN,arquimea ingenieria,arquimea ingenieria,0.0,ARQUIMEA ENTERPRISES SL,None,gleif,"""ARQUIMEA",
...,...,...,...,...,...,...,...,...,...,...
4890,TFN PROPROTÃÂ,NaN,tfs,None,14.0,VIEL ET COMPAGNIE-FINANCE,None,None,TFN,TFN
4891,UK AND IRELAND SAP USER GROUP LTD,NaN,uk as,uk assistance,42.0,DIRECT LINE INSURANCE GROUP PLC,None,None,UK,
4892,VSW VERBAND FUER SICHERHEIT IN DER WIRTSCHAFT,NaN,vsws,vswsl masterfonds,64.0,UI BVK Kapitalverwaltungsgesellschaft mbH,None,None,VSW,
4893,WILLI EBERLEIN GMBH,NaN,william,None,11.0,SOCIETE GENERALE,None,None,WILLI,


In [413]:
df_cluster_clean

,Raw name,Mapped name,Closest node,Closest company,Distance to closest,Closest parent,Confidence score,What parent,First word,LCS clustering parent
0,"""ACCESOS NORMALIZADOS, SL""",NaN,accesor,accesor aplications and services,88.0,JCM TECHNOLOGIES SA,None,None,"""ACCESOS",
1,"""ALTAIX ELECTRONICA , S.A.L.""",NaN,altair eu,altair european opportunities fi,136.0,"ALTAIR FINANCE ASSET MANAGEMENT SGIIC, SA",None,None,"""ALTAIX",
2,"""ANTALA LOCKS & ACCESORIS, SL""",NaN,antalia,antalia,19.0,JAD,None,None,"""ANTALA",
3,"""ANTERAL, SL""",NaN,antero,antero peak master fund lp,43.0,ARTISAN PARTNERS LIMITED PARTNERSHIP,None,None,"""ANTERAL,",
4,"""ARQUIMEA INGENIERIA , S.L.""",NaN,arquimea ingenieria,arquimea ingenieria,0.0,ARQUIMEA ENTERPRISES SL,None,gleif,"""ARQUIMEA",
...,...,...,...,...,...,...,...,...,...,...
4890,TFN PROPROTÃÂ,NaN,tfs,None,14.0,VIEL ET COMPAGNIE-FINANCE,None,None,TFN,TFN
4891,UK AND IRELAND SAP USER GROUP LTD,NaN,uk as,uk assistance,42.0,DIRECT LINE INSURANCE GROUP PLC,None,None,UK,
4892,VSW VERBAND FUER SICHERHEIT IN DER WIRTSCHAFT,NaN,vsws,vswsl masterfonds,64.0,UI BVK Kapitalverwaltungsgesellschaft mbH,None,None,VSW,
4893,WILLI EBERLEIN GMBH,NaN,william,None,11.0,SOCIETE GENERALE,None,None,WILLI,


In [416]:
def output_final_result(df):
    """Fills the 'Mapped name' row with the precedently matched parent organization name, and writes the resulting table with only the relevant colums in `output.csv` """
    def update_row(row):
        if row['What parent'] == 'gleif':
            row['Mapped name'] = row['Closest parent']
        elif row['What parent'] == 'cluster':
            row['Mapped name'] = row['LCS clustering parent']
        else:
            row['Mapped name'] = row['Raw name']
        return row

    df = df.apply(update_row, axis=1)

    output_df = df[['Raw name', 'Mapped name']]

    output_df.to_csv('output.csv', index=False)

    return output_df

output_final_result(df_cluster_clean)

,Raw name,Mapped name
0,"""ACCESOS NORMALIZADOS, SL""","""ACCESOS NORMALIZADOS, SL"""
1,"""ALTAIX ELECTRONICA , S.A.L.""","""ALTAIX ELECTRONICA , S.A.L."""
2,"""ANTALA LOCKS & ACCESORIS, SL""","""ANTALA LOCKS & ACCESORIS, SL"""
3,"""ANTERAL, SL""","""ANTERAL, SL"""
4,"""ARQUIMEA INGENIERIA , S.L.""",ARQUIMEA ENTERPRISES SL
...,...,...
4890,TFN PROPROTÃÂ,TFN PROPROTÃÂ
4891,UK AND IRELAND SAP USER GROUP LTD,UK AND IRELAND SAP USER GROUP LTD
4892,VSW VERBAND FUER SICHERHEIT IN DER WIRTSCHAFT,VSW VERBAND FUER SICHERHEIT IN DER WIRTSCHAFT
4893,WILLI EBERLEIN GMBH,WILLI EBERLEIN GMBH
